Importazione dataset e correzione anomalie nei dati

In [18]:
import pandas as pd

In [19]:
gps=pd.read_csv("C:\\Users\\franc\\Desktop\\progettofondamenti\\googleplaystore.csv")
gps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [20]:
gpsu=pd.read_csv("C:\\Users\\franc\\Desktop\\progettofondamenti\\googleplaystore_user_reviews.csv")
gpsu.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [4]:
#Per prima cosa facciamo un groupby di una variabile per controllare eventuali anomalie
gps.groupby('Category').size() # Una Category si chiama "1.9", qualcosa non va, vediamo a che riga corrisponde

Category
1.9                       1
ART_AND_DESIGN           65
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     231
BUSINESS                460
COMICS                   60
COMMUNICATION           387
DATING                  234
EDUCATION               156
ENTERTAINMENT           149
EVENTS                   64
FAMILY                 1972
FINANCE                 366
FOOD_AND_DRINK          127
GAME                   1144
HEALTH_AND_FITNESS      341
HOUSE_AND_HOME           88
LIBRARIES_AND_DEMO       85
LIFESTYLE               382
MAPS_AND_NAVIGATION     137
MEDICAL                 463
NEWS_AND_MAGAZINES      283
PARENTING                60
PERSONALIZATION         392
PHOTOGRAPHY             335
PRODUCTIVITY            424
SHOPPING                260
SOCIAL                  295
SPORTS                  384
TOOLS                   843
TRAVEL_AND_LOCAL        258
VIDEO_PLAYERS           175
WEATHER                  82
dtype: int64

In [5]:
gps.index[gps["Category"]=="1.9"] #Corrisponde alla riga 10472, forse c'è qualche anomalia in corrispondenza di quella riga

Int64Index([10472], dtype='int64')

In [6]:
gps.iloc[10472] #Manca un NaN in corrispondenza della Category, ciò fa traslare tutti i valori, bisogna sistemare

App               Life Made WI-Fi Touchscreen Photo Frame
Category                                              1.9
Rating                                                 19
Reviews                                              3.0M
Size                                               1,000+
Installs                                             Free
Type                                                    0
Price                                            Everyone
Content Rating                                        NaN
Genres                                  February 11, 2018
Last Updated                                       1.0.19
Current Ver                                    4.0 and up
Android Ver                                           NaN
Name: 10472, dtype: object

In [21]:
for i in range(1,12):
    gps.iloc[10472,-i]=gps.iloc[10472,-i-1]
gps.iloc[10472,1]="PHOTOGRAPHY"
gps.iloc[10472] #Ora la riga 10472 è stata sistemata (ha senso il fatto che, mancando inizialmente la Category, manchi il genere)

App               Life Made WI-Fi Touchscreen Photo Frame
Category                                      PHOTOGRAPHY
Rating                                                1.9
Reviews                                                19
Size                                                 3.0M
Installs                                           1,000+
Type                                                 Free
Price                                                   0
Content Rating                                   Everyone
Genres                                                NaN
Last Updated                            February 11, 2018
Current Ver                                        1.0.19
Android Ver                                    4.0 and up
Name: 10472, dtype: object

PRIMA PARTE

1.1 Convert the app sizes to a number

In [8]:
gps.groupby('Size').size() # guardiamo i diversi valori di 'Size'

Size
1.0M                     7
1.1M                    32
1.2M                    41
1.3M                    35
1.4M                    37
1.5M                    48
1.6M                    39
1.7M                    40
1.8M                    50
1.9M                    32
10.0M                   10
100M                    16
1020k                    1
103k                     1
108k                     1
10M                    136
116k                     1
118k                     3
11M                    198
11k                      1
121k                     1
122k                     1
12M                    196
13M                    191
141k                     2
143k                     1
144k                     1
14M                    194
14k                      1
153k                     1
                      ... 
939k                     1
93M                     16
93k                      1
940k                     1
942k                     1
948k                   

In [22]:
#Bisogna trasformare i caratteri k e M
for i in gps.index:
    if 'k' in str(gps.loc[i,'Size']):
        gps.loc[i,'Size']=str(gps.loc[i,'Size'])[:-1]
        gps.loc[i,'Size']=float(gps.loc[i,'Size'])*1000
    elif 'M' in str(gps.loc[i,'Size']):
        gps.loc[i,'Size']=str(gps.loc[i,'Size'])[:-1]
        gps.loc[i,'Size']=float(gps.loc[i,'Size'])*1000000
    elif (str(gps.loc[i,'Size'])[-1])  in ['0','1','2','3','4','5','6','7','8','9']: # non possiamo trasformare in interi i valori 'Varies with device'
        gps.loc[i,'Size']=float(gps.loc[i,'Size'])
gps.groupby('Size').size() # Ora sembrano corretti   

Size
8500.0                   1
11000.0                  1
14000.0                  1
17000.0                  2
18000.0                  2
20000.0                  1
23000.0                  1
24000.0                  1
25000.0                  1
26000.0                  2
27000.0                  1
28000.0                  1
29000.0                  2
33000.0                  2
34000.0                  1
39000.0                  1
41000.0                  1
44000.0                  1
45000.0                  1
48000.0                  1
50000.0                  1
51000.0                  2
54000.0                  1
55000.0                  1
58000.0                  2
61000.0                  1
67000.0                  1
70000.0                  2
72000.0                  2
73000.0                  1
                      ... 
72000000.0              19
73000000.0              15
74000000.0              20
75000000.0              13
76000000.0              27
77000000.0             

1.2 Convert the number of installs to a number

In [10]:
gps.groupby('Installs').size()#diamo un'occhiata ai diversi valori di installs

Installs
0                    1
0+                  14
1+                  67
1,000+             908
1,000,000+        1579
1,000,000,000+      58
10+                386
10,000+           1054
10,000,000+       1252
100+               719
100,000+          1169
100,000,000+       409
5+                  82
5,000+             477
5,000,000+         752
50+                205
50,000+            479
50,000,000+        289
500+               330
500,000+           539
500,000,000+        72
dtype: int64

In [23]:
#Bisogna togliere le virgole e i + e trasformarli in valori numerici
for i in range(0, len(gps.index)):
    carat=gps.loc[i,"Installs"]
    l=list(carat)
    h=[]
    for el in l:
        if el is not "," and el is not "+":
            h.append(el)
    gps.loc[i,"Installs"]="".join(h)
    gps.loc[i,'Installs']=int(gps.loc[i,'Installs'])
gps.groupby('Installs').size() #Ora sembrano corretti

Installs
0               15
1               67
5               82
10             386
50             205
100            719
500            330
1000           908
5000           477
10000         1054
50000          479
100000        1169
500000         539
1000000       1579
5000000        752
10000000      1252
50000000       289
100000000      409
500000000       72
1000000000      58
dtype: int64

1.3 Transform “Varies with device” into a missing value

In [24]:
for j in range (0,len(gps.columns)):
    for i in range(0,len(gps.index)):
        if gps.iloc[i,j]=="Varies with device":          
                gps.iloc[i,j]=None
gps.head() #Current Ver della riga di indice 3 era "Varies with Device", ora è "None"

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,1.9e+07,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,1.4e+07,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7e+06,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,2.5e+07,50000000,Free,0,Teen,Art & Design,"June 8, 2018",None,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8e+06,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


1.4 Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

In [13]:
gps.groupby('Android Ver').size()

Android Ver
1.0 and up          2
1.5 and up         20
1.6 and up        116
2.0 and up         32
2.0.1 and up        7
2.1 and up        134
2.2 - 7.1.1         1
2.2 and up        244
2.3 and up        652
2.3.3 and up      281
3.0 and up        241
3.1 and up         10
3.2 and up         36
4.0 and up       1376
4.0.3 - 7.1.1       2
4.0.3 and up     1501
4.1 - 7.1.1         1
4.1 and up       2451
4.2 and up        394
4.3 and up        243
4.4 and up        980
4.4W and up        12
5.0 - 6.0           1
5.0 - 7.1.1         1
5.0 - 8.0           2
5.0 and up        601
5.1 and up         24
6.0 and up         60
7.0 - 7.1.1         1
7.0 and up         42
7.1 and up          3
8.0 and up          6
dtype: int64

In [25]:
import re
for j in ["Android Ver","Current Ver"]:
    for i in gps.index:
        if re.search(r'\d+[.][.|\d]*',str(gps.loc[i,j])):
            gps.loc[i,j]=re.findall(r'\d+[.][.|\d]*',str(gps.loc[i,j]))[-1] 
#[-1] perchè prende il penultimo elemento della lista, ovvero il secondo nel caso in cui ci siano due versioni separate da "-",
# il primo nel caso in cui vi sia un solo elemento

In [15]:
gps.groupby('Android Ver').size() #Ora sembrano sistemati          

Android Ver
1.0         2
1.5        20
1.6       116
2.0        32
2.0.1       7
2.1       134
2.2       244
2.3       652
2.3.3     281
3.0       241
3.1        10
3.2        36
4.0      1376
4.0.3    1501
4.1      2451
4.2       394
4.3       243
4.4       992
5.0       601
5.1        24
6.0        61
7.0        42
7.1         3
7.1.1       6
8.0         8
dtype: int64

In [16]:
gps.groupby('Current Ver').size() #Alcune "Current Ver" sono intrattabili

Current Ver
0.0.0.2                                     1
0.0.1                                      15
0.0.10                                      1
0.0.2                                       4
0.0.3                                       2
0.0.4                                       3
0.0.42                                      1
0.0.44                                      1
0.0.5                                       3
0.0.52                                      1
0.0.53                                      1
0.0.6                                       1
0.0.7                                       3
0.0.73                                      1
0.0.80                                      1
0.0.9                                       5
0.1                                        13
0.1.0                                       2
0.1.1                                       3
0.1.10.0                                    1
0.1.100944346                               2
0.1.11                

1.5 Remove the duplicates

In [26]:
gps.drop_duplicates()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,1.9e+07,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3
1,Coloring book moana,ART_AND_DESIGN,3.9,967,1.4e+07,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7e+06,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,2.5e+07,50000000,Free,0,Teen,Art & Design,"June 8, 2018",None,4.2
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8e+06,100000,Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6e+06,50000,Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,1.9e+07,50000,Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,2.9e+07,1000000,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,3.3e+07,1000000,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1e+06,10000,Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3


1.6 For each category, compute the number of apps

In [27]:
gps.groupby('Category')['App'].count()

Category
ART_AND_DESIGN           65
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     231
BUSINESS                460
COMICS                   60
COMMUNICATION           387
DATING                  234
EDUCATION               156
ENTERTAINMENT           149
EVENTS                   64
FAMILY                 1972
FINANCE                 366
FOOD_AND_DRINK          127
GAME                   1144
HEALTH_AND_FITNESS      341
HOUSE_AND_HOME           88
LIBRARIES_AND_DEMO       85
LIFESTYLE               382
MAPS_AND_NAVIGATION     137
MEDICAL                 463
NEWS_AND_MAGAZINES      283
PARENTING                60
PERSONALIZATION         392
PHOTOGRAPHY             336
PRODUCTIVITY            424
SHOPPING                260
SOCIAL                  295
SPORTS                  384
TOOLS                   843
TRAVEL_AND_LOCAL        258
VIDEO_PLAYERS           175
WEATHER                  82
Name: App, dtype: int64

1.7 For each category, compute the average rating

In [28]:
for i in range(0,len(gps.index)): #alcuni valori sono delle strighe , quindi dobbiamo trasformare tutto in float
    gps.iloc[i,2]=float(gps.iloc[i,2])  

In [29]:
gps.groupby('Category')['Rating'].sum()/gps.groupby('Category')['Rating'].size()

Category
ART_AND_DESIGN         4.156923
AUTO_AND_VEHICLES      3.598824
BEAUTY                 3.390566
BOOKS_AND_REFERENCE    3.348918
BUSINESS               2.714783
COMICS                 4.016667
COMMUNICATION          3.524548
DATING                 3.308974
EDUCATION              4.360897
ENTERTAINMENT          4.126174
EVENTS                 3.118750
FAMILY                 3.713945
FINANCE                3.646448
FOOD_AND_DRINK         3.576378
GAME                   4.110227
HEALTH_AND_FITNESS     3.725220
HOUSE_AND_HOME         3.625000
LIBRARIES_AND_DEMO     3.195294
LIFESTYLE              3.365969
MAPS_AND_NAVIGATION    3.667153
MEDICAL                3.166739
NEWS_AND_MAGAZINES     3.402120
PARENTING              3.583333
PERSONALIZATION        3.473214
PHOTOGRAPHY            3.960714
PRODUCTIVITY           3.486321
SHOPPING               3.899231
SOCIAL                 3.736271
SPORTS                 3.508594
TOOLS                  3.524081
TRAVEL_AND_LOCAL       3.599612

1.8 Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [30]:
# creiamo il primo dataframe , solo con i generi
Genress=pd.DataFrame(gps.groupby('Genres').size(),columns=['frequency'])
se=pd.Series(Genress.index)
g=list(se)
h=[]
j=[]
for el in g:
    l=list(el)
    if ";" in l:
        j.append(el)
    if ";" not in l:
        h.append(el)
for parola in j:
    lista=parola.split(";")
    for gen in lista:
        if gen not in h:
            h.append(gen)
hh=pd.DataFrame(h,columns=['genere'])

In [31]:
hh #Dataframe richiesto

,genere
0,Action
1,Adventure
2,Arcade
3,Art & Design
4,Auto & Vehicles
5,Beauty
6,Board
7,Books & Reference
8,Business
9,Card


In [32]:
#creiamo una lista di liste , ognuna della quale comprende tutte le applicazini per un diverso genere
l=[]
for j in hh.index:
    g=[]
    for i in gps.index:
        if str(hh['genere'][j])  in str(gps.iloc[i,9]).split(';'):
             g.append(gps.iloc[i,0])
    l.append(g)
    

In [34]:
#creiamo un dataframe in cui abbiamo le applicazioni per genere
g=pd.Series(l,index=h)
gg=pd.DataFrame(g,columns=['App'])
gg


,App
Action,"[slither.io, Temple Run 2, Helix Jump, Zombie ..."
Adventure,"[ROBLOX, ROBLOX, ROBLOX, Pokémon GO, Harry Pot..."
Arcade,"[Subway Surfers, Angry Birds Rio, Sonic Dash, ..."
Art & Design,[Photo Editor & Candy Camera & Grid & ScrapBoo...
Auto & Vehicles,"[Monster Truck Stunt 3D 2019, Real Tractor Far..."
Beauty,"[Hush - Beauty for Everyone, ipsy: Makeup, Bea..."
Board,"[Color by Number – New Coloring Book, New YAHT..."
Books & Reference,"[Wattpad 📖 Free Books, E-Book Read - Read Book..."
Business,"[Visual Voicemail by MetroPCS, Indeed Job Sear..."
Card,"[Solitaire, Solitaire TriPeaks, Solitaire, Sol..."


1.9 For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

In [35]:
#aggiungiamo colonne , ognuna delle quali si riferisce a un genere diverso, ogni colonna è composta di soli false
for i in hh.genere :
    gps.insert(13,i,False)

In [36]:
# diamo valori true , se l'applicazione ha quel genere 
for elem in gps.index:
    for col in range(13,65):
        if gps.columns[col] in str(gps.loc[elem,'Genres']).split(';'):
            gps.iloc[elem,col]=True

In [37]:
gps.head() #sembra essere giusto, risolto anche problema dei generi separati da ";"

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Card,Business,Books & Reference,Board,Beauty,Auto & Vehicles,Art & Design,Arcade,Adventure,Action
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,1.9e+07,10000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,True,False,False,False
1,Coloring book moana,ART_AND_DESIGN,3.9,967,1.4e+07,500000,Free,0,Everyone,Art & Design;Pretend Play,...,False,False,False,False,False,False,True,False,False,False
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7e+06,5000000,Free,0,Everyone,Art & Design,...,False,False,False,False,False,False,True,False,False,False
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,2.5e+07,50000000,Free,0,Teen,Art & Design,...,False,False,False,False,False,False,True,False,False,False
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8e+06,100000,Free,0,Everyone,Art & Design;Creativity,...,False,False,False,False,False,False,True,False,False,False


1.10 For each genre, compute the average rating. What is the genre with highest average?

In [38]:
gr=[]
for j in gps.columns[13:]:
    gr.append(gps[gps[j]==True].groupby(j)['Rating'].sum()/gps[gps[j]==True].groupby(j)['Rating'].size())
a=pd.DataFrame(gr,index=gps.columns[13:]) #troviamo la media dei rating per genere

In [39]:
a.idxmax() #troviamo l'osservazione con rating medio maggiore

True    Brain Games
dtype: object

In [40]:
gps.groupby('Price').size()

Price
$0.99        148
$1.00          3
$1.04          1
$1.20          1
$1.26          1
$1.29          1
$1.49         46
$1.50          1
$1.59          1
$1.61          1
$1.70          2
$1.75          1
$1.76          1
$1.96          1
$1.97          1
$1.99         73
$10.00         3
$10.99         2
$109.99        1
$11.99         5
$12.99         5
$13.99         2
$14.00         1
$14.99        11
$15.46         1
$15.99         1
$154.99        1
$16.99         3
$17.99         2
$18.99         1
           ...  
$389.99        1
$39.99         2
$394.99        1
$399.99       12
$4.29          1
$4.49          9
$4.59          1
$4.60          1
$4.77          1
$4.80          1
$4.84          1
$4.85          1
$4.99         72
$400.00        1
$46.99         1
$5.00          1
$5.49          6
$5.99         30
$6.49          5
$6.99         13
$7.49          2
$7.99          7
$74.99         1
$79.99         2
$8.49          2
$8.99          5
$89.99         1
$9.00   

1.11 For each app, compute the approximate income, obtain as a product of number of installs and price.

In [41]:
for i in gps.index:                   
    Prezzo=str(gps.loc[i,'Price'])
    Pr=list(Prezzo)
    if '$' in Pr:       #abbiamo notato che '$' era presente in ogni osservazione di price e quindi abbiamo dovuto rimuoverlo per poter convertire ogni prezzo in float
        Pr.remove('$')
    gps.loc[i,'Price']=float(''.join(Pr))
gps["Income"] = gps.apply(lambda row: float(row.Price) * float(row.Installs), axis=1)
gps.groupby("Income").size()    

Income
0.00           10051
0.99               8
1.49               4
1.99               1
2.49               1
2.99               2
3.99               1
4.95               5
5.99               2
7.45               1
9.90              24
9.95               2
14.90              5
19.60              1
19.90              2
20.00              1
24.90              3
28.99              1
29.90              7
29.95              2
30.40              1
34.90              1
36.10              1
44.95              1
48.00              1
49.50              8
49.90              3
52.00              1
54.90              3
59.90              5
               ...  
900000.00          2
990000.00          9
995000.00          2
999000.00          4
999500.00          1
1199000.00         1
1245000.00         1
1399000.00         1
1495000.00         4
1499000.00         1
1499500.00         1
1990000.00         2
1995000.00         2
1999950.00         3
2490000.00         2
2495000.00         1
249900

1.12 For each app, compute its minimum and maximum Sentiment_polarity

In [31]:
#troviamo minimo
gpsu.groupby('App')['Sentiment_Polarity'].min()

App
10 Best Foods for You                                -0.800000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                     -0.112500
11st                                                 -1.000000
1800 Contacts - Lens Store                           -0.300000
1LINE – One Line with One Touch                      -0.825000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif     -0.800000
21-Day Meditation Experience                         -0.265625
2Date Dating App, Love and matching                  -0.645833
2GIS: directory & navigator                          -0.375000
2RedBeans                                            -0.800000
2ndLine - Second Phone Number                        -0.781250
30 Day Fitness Challenge - Workout at Home           -0.500000
365Scores - Live Scores                               0.000000
3D Blue Glass Water Keyboard Theme                         NaN
3D Color Pixel by Number - Sandbox Art Coloring            NaN
3D Live Neon Weed Launcher                         

In [32]:
#troviamo massimo
gpsu.groupby('App')['Sentiment_Polarity'].max()

App
10 Best Foods for You                                 1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                      0.910000
11st                                                  1.000000
1800 Contacts - Lens Store                            0.838542
1LINE – One Line with One Touch                       1.000000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif      1.000000
21-Day Meditation Experience                          0.587500
2Date Dating App, Love and matching                   1.000000
2GIS: directory & navigator                           1.000000
2RedBeans                                             1.000000
2ndLine - Second Phone Number                         1.000000
30 Day Fitness Challenge - Workout at Home            0.900000
365Scores - Live Scores                               1.000000
3D Blue Glass Water Keyboard Theme                         NaN
3D Color Pixel by Number - Sandbox Art Coloring            NaN
3D Live Neon Weed Launcher                         

SECONDA PARTE

2.1 For each app, compute the average number of words in its reviews

In [42]:
# creiamo una variabile 'numero-parole' che conta le parole presenti nella variabile 'Translated_Review'
for i in range(0,len(gpsu.index)): 
    tr=str(gpsu.iloc[i,1])
    if tr=='nan':
        gpsu.loc[i,'numero-parole']=None
    else:
        ltr=tr.split(" ")
        c=len(ltr)
        gpsu.loc[i,'numero-parole']=int(c)
 


In [43]:
#ora calcoliamo la media delle parole usate per applicazione
gpsu.groupby('App')['numero-parole'].mean()

App
10 Best Foods for You                                  6.659794
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室                       6.275000
11st                                                  12.487179
1800 Contacts - Lens Store                             9.250000
1LINE – One Line with One Touch                        3.947368
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif       5.468750
21-Day Meditation Experience                          24.112500
2Date Dating App, Love and matching                   13.157895
2GIS: directory & navigator                           18.075000
2RedBeans                                              5.307692
2ndLine - Second Phone Number                         11.250000
30 Day Fitness Challenge - Workout at Home            15.612903
365Scores - Live Scores                               10.714286
3D Blue Glass Water Keyboard Theme                          NaN
3D Color Pixel by Number - Sandbox Art Coloring             NaN
3D Live Neon Weed Launcher          

2.2 For each app, compute its longest review

In [45]:
ggg=gpsu.loc[gpsu.groupby('App')['numero-parole'].idxmax()]
ggg.iloc[:len(ggg.index) ,:2] #agli indici la lunghezza della Review più lunga per applicazione, a noi interessa quale sia questa Review, ovvero la seconda colonna

C:\Users\franc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,App,Translated_Review
37.0,10 Best Foods for You,WEALTH OF INFORMATION Very informative ... eas...
224.0,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,Is the development method completed in the ear...
271.0,11st,Open Market Revolution A section of the Helo-S...
311.0,1800 Contacts - Lens Store,Good prices exceptional customer service. Orde...
364.0,1LINE – One Line with One Touch,Love game deleting I've waiting 3 months level...
404.0,2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,"BullSh*t.... When open apps, says sill never r..."
443.0,21-Day Meditation Experience,I guess can't please everyone... I loved updat...
525.0,"2Date Dating App, Love and matching","I like dating much. Different dating apps, mem..."
592.0,2GIS: directory & navigator,"For pedestrians, especially in a foreign city,..."
618.0,2RedBeans,I really like could help find future Chinese h...


2.3 For each app, compute the ratio between the number of installs and the number of reviews

In [36]:
d1=pd.DataFrame(gps.groupby('App')['Installs'].sum())
d2=pd.DataFrame(gps.groupby('App')['Reviews'].sum())
d=pd.concat([d1,d2],axis=1)
for i in d.index:
    if float(d.loc[i,"Reviews"])==0:
        d.loc[i,"Ratio"]=None 
    else:
        d.loc[i,"Ratio"]=float(d.loc[i,"Installs"])/float(d.loc[i,"Reviews"])
d

,Installs,Reviews,Ratio
App,,,
"""i DT"" Fútbol. Todos Somos Técnicos.",500,27,18.518519
+Download 4 Instagram Twitter,1000000,40467,24.711493
- Free Comics - Comic Apps,10000,115,86.956522
.R,10000,259,38.610039
/u/app,10000,573,17.452007
058.ba,100,27,3.703704
1. FC Köln App,100000,2019,49.529470
10 Best Foods for You,1000000,24902490,0.040157
10 Minutes a Day Times Tables,100000,681,146.842878


2.4 Cluster the apps according to the major android version (the first two digits — e.g. for 4.0.3 the major version is 4.0)

In [47]:
gps['new-androidver']=gps['Android Ver'] #creiamo una nuova variabile uguale ad 'Android Ver'

In [48]:
for i in gps.index:
    gps.loc[i,'new-androidver']=str(gps.loc[i,'Android Ver'])[0:3]  #I valori che erano none(varies with device,vedi punto 3), 
    #sono diventati'non'; concettualmente non cambia nulla, ha semplicemente preso i primi 3 caratteri

In [49]:
gps.groupby('new-androidver').size()  

new-androidver
1.0       2
1.5      20
1.6     116
2.0      39
2.1     134
2.2     244
2.3     933
3.0     241
3.1      10
3.2      36
4.0    2877
4.1    2451
4.2     394
4.3     243
4.4     992
5.0     601
5.1      24
6.0      61
7.0      42
7.1       9
8.0       8
Non    1362
nan       2
dtype: int64

In [50]:
clust=pd.DataFrame(gps.groupby('new-androidver').size())

In [51]:
lislis=[] #lista di liste
for j in clust.index:  
    listaapp=[]
    for i in gps.index:
        if str(gps.loc[i,"new-androidver"])==j:
            listaapp.append(gps.loc[i,"App"])
    lislis.append(listaapp)
clust["ListaApp"]=lislis
clust

,0,ListaApp
new-androidver,,
1.0,2,"[Bible du Semeur-BDS (French), Tamilnadu Elect..."
1.5,20,"[Cool Reader, SuperLivePro, Wifi Connect Libra..."
1.6,116,"[Dresses Ideas & Fashions +3000, English speak..."
2.0,39,"[GO Notifier, Prosperity, Smart Launcher theme..."
2.1,134,"[Flybook, ExDialer - Dialer & Contacts, Rocket..."
2.2,244,"[Restart Navigator, Azpen eReader, Plugin:AOT ..."
2.3,933,"[Paper flowers instructions, 3D Color Pixel by..."
3.0,241,"[Garden Coloring Book, Harley Quinn wallpapers..."
3.1,10,"[Photo Effects Pro, I am Nose Doctor -Save my ..."


2.5 For each cluster, compute the average date and the last date of an update.

In [52]:
gps['a'] = pd.to_datetime(gps['Last Updated'],format='%B %d, %Y') #creiamo variabile a in cui abbiamo 'Last Updated' in formato data

In [53]:
#creiamo lista in cui abbiamo la media delle date per ogni cluster
datemedie=[]
for j in clust.index:
    datamedia=(gps[gps['new-androidver']==j].a-gps[gps['new-androidver']==j].a.min()).mean()+gps[gps['new-androidver']==j].a.min()
    datemedie.append(datamedia)
datemedie

[Timestamp('2016-09-21 00:00:00'),
 Timestamp('2015-01-14 21:36:00'),
 Timestamp('2015-11-11 08:04:08.275862080'),
 Timestamp('2015-06-14 17:13:50.769230768'),
 Timestamp('2015-05-23 22:34:01.791044784'),
 Timestamp('2015-09-25 21:44:15.737704912'),
 Timestamp('2017-01-04 19:03:39.935691328'),
 Timestamp('2016-11-16 09:27:38.091286304'),
 Timestamp('2015-08-25 09:36:00'),
 Timestamp('2017-02-22 15:20:00'),
 Timestamp('2017-12-03 06:20:23.774765376'),
 Timestamp('2018-03-20 17:58:40.685434512'),
 Timestamp('2018-02-19 11:41:43.553299488'),
 Timestamp('2018-02-09 18:57:46.666666672'),
 Timestamp('2018-04-23 21:10:09.677419360'),
 Timestamp('2018-05-08 14:27:21.264559072'),
 Timestamp('2018-02-17 14:00:00'),
 Timestamp('2018-04-02 22:49:10.819672128'),
 Timestamp('2018-05-08 20:00:00'),
 Timestamp('2018-03-14 16:00:00'),
 Timestamp('2018-05-29 09:00:00'),
 Timestamp('2018-04-22 02:56:33.832599104'),
 Timestamp('2018-05-23 12:00:00')]

In [54]:
clust['mediedate']=datemedie  # aggiungiamo una colonna 'mediedate' che rappresenta la medie delle date ottenute nel punto precedente

In [55]:
gps.groupby('new-androidver')['a'].max() #troviamo max per ogni cluster

new-androidver
1.0   2018-08-04
1.5   2018-08-04
1.6   2018-08-06
2.0   2018-07-09
2.1   2018-07-21
2.2   2018-08-03
2.3   2018-08-07
3.0   2018-08-02
3.1   2017-05-24
3.2   2018-07-28
4.0   2018-08-08
4.1   2018-08-08
4.2   2018-08-08
4.3   2018-08-07
4.4   2018-08-07
5.0   2018-08-08
5.1   2018-08-01
6.0   2018-08-04
7.0   2018-08-04
7.1   2018-07-12
8.0   2018-08-03
Non   2018-08-08
nan   2018-07-20
Name: a, dtype: datetime64[ns]

2.6 Excluding the free apps, what is the content rating with highest average price?

In [56]:
pm=gps[gps.Price >0]

In [60]:
pm2=pd.DataFrame(pm.groupby('Content Rating')['Price'].sum()/pm.groupby('Content Rating')['Price'].size())

In [66]:
pm2

,Price
Content Rating,
Everyone,14.684317
Everyone 10+,4.748182
Mature 17+,5.905000
Teen,12.620769


In [63]:
pm2.idxmax()

Price    Everyone
dtype: object